# **Stock Sentiment**

## 0. Imports

In [ ]:
import os
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

import gensim.downloader

os.chdir("../scripts")
from constants import TRAIN_DATA,  TEST_DATA, LABELS
from aux_funcs import preprocess, find_punctuated_tokens, eval_sklearn_model, eval_lstm_model, eval_transformer, eval_llm_model
os.chdir("../classes")
from word2vec import Word2Vec

In [ ]:
# # Download NLTK resources
# nltk.download('stopwords')
# nltk.download('wordnet')

## 1. Data Loading

In [ ]:
train_data = pd.read_csv(TRAIN_DATA)
test_data = pd.read_csv(TEST_DATA)

In [ ]:
train_data.head()

In [ ]:
test_data.head()

## 2. Data Exploration

In [ ]:
# Checking for missing values
train_data.isna().sum()

There are no missing values in the dataset, so it is safe to proceed.

### 2.1. Text Feature

#### 2.1.1. Word Count

Start by spliting `text` into tokens and analysing the number of tokens in each document.

In [ ]:
# Fetch wordcount for each document
train_data['word_count']  = train_data['text'].apply(lambda x: len(str(x).split(" ")))
train_data[['text','word_count']].head()

#### 2.1.2. Descriptive Statistics

In [ ]:
train_data['word_count'].describe()

In [ ]:
#Plot distribution of text length
sns.histplot(train_data['word_count'], bins=30)
plt.title("Distribution of Text Lengths (raw text)")
plt.xlabel("Length of text")
plt.ylabel("Frequency")
plt.show()

Considering the table and plot above, it is evident that some rows have very few words. Below, is a deeper exploration of these cases.

In [ ]:
# Checking records with 3 words or less
train_data[train_data['word_count'] <= 3]

Looking at the records with 3 or less words, most belong to the Neutral category (label = 2). Since there is an overrepresentation of this class in the dataset, below the records above belonging to the Neutral category are excluded.

In [ ]:
train_data = train_data[~((train_data['word_count'] <= 3) & (train_data['label'] == 2))]

In [ ]:
# Checking word frequency
all_words = ' '.join(train_data['text']).split()
freq = pd.Series(all_words).value_counts()
freq.sort_values(ascending=False)[0:10]

From the table above, "the" and "The" are both words in the top 10 - this could indicate a beneficial effect of lowercasing the 'text' feature.<br>
Also, "-" is a very common symbol which has no meaning and should be removed from the feature.

In [ ]:
# Plotting the word frequencies count
x_labels = freq.index[0:10]
values = freq[:10]
plt.bar(x_labels, values, align='center', alpha=0.5)
plt.xticks(x_labels)
plt.ylabel('Frequencies')
plt.title('Words')

plt.show()

In [ ]:
# Create a word cloud from the text feature
raw_text = ' '.join(train_data['text'].fillna('')).lower()
raw_wc = WordCloud(width=800, height=400, background_color='white').generate(raw_text)
plt.imshow(raw_wc, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud (Raw Text)')
plt.show()

### 2.2. Target Feature

In [ ]:
train_data['label'].value_counts()

In [ ]:
# Visualize distribution of label
sns.countplot(data=train_data, x='label')
plt.title("Distribution of label")
plt.xlabel("label")
plt.ylabel("Count")
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(y='label', data=train_data, width=0.3)

plt.title('Boxplot of Label', fontsize=16)
plt.ylabel('Label', fontsize=12)
plt.xlabel('')

plt.show()

## 3. Data Preprocessing

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
# Defining stop words
stop = set(stopwords.words('english'))
# Initializing Lemmatizer and Stemmer
lemma = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

In [ ]:
train_data['text'] = train_data['text'].apply(
    lambda x: preprocess(
        text=x
        ,keep_ticker=False
        ,keep_url=False
        ,stopwords=stop
        ,lemmatizer=lemma
    )
)

In [ ]:
test_data['text'] = test_data['text'].apply(
    lambda x: preprocess(
        text=x
        ,keep_ticker=False
        ,keep_url=False
        ,stopwords=stop
        ,lemmatizer=lemma
    )
)

### 3.1. Rechecking the Plots

In [ ]:
all_words = ' '.join(train_data['text']).split()
freq = pd.Series(all_words).value_counts()
freq.sort_values(ascending=False)[0:10]

In [ ]:
plt.figure(figsize=(10, 5))
x_labels = freq.index[0:10]
values = freq[:10]

plt.bar(x_labels, values, align='center', alpha=0.5)
plt.ylabel('Frequencies')
plt.title('Top 10 Words')
plt.tight_layout()

plt.show()

In [ ]:
# Drop rows with encoding errors
exceptions = find_punctuated_tokens(train_data['text'])

pattern = r'(?:' + '|'.join(re.escape(word) for word in exceptions) + r')'

train_data = train_data[~train_data['text'].str.contains(pattern, case=False, na=False)]

In [ ]:
# Word cloud
raw_text = ' '.join(train_data['text'].fillna('')).lower()
raw_wc = WordCloud(width=800, height=400, background_color='white').generate(raw_text)
plt.imshow(raw_wc, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud (Raw Text)')
plt.show()

In [ ]:
# Define dependent and independent features
# Train
X_train = train_data['text']
y_train = train_data['label']
# Test
X_test = test_data['text']

## 4. Feature Engineering

In [ ]:
# Bag of Words
bow = CountVectorizer(binary=True)  # using binary=True, because working with short text

In [ ]:
# Word2Vec
emb_size = 100
w2v_model = gensim.downloader.load(f'glove-twitter-{emb_size}')
w2v = Word2Vec(w2v_model)

In [ ]:
# Transformer
transf_model = "cardiffnlp/twitter-roberta-base-sentiment"

## 5. Modelling

In [ ]:
# Stratified K-Fold
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=20)

### 5.1. KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=10, metric='cosine', weights='distance')

#### Bag of Words

In [ ]:
# Evaluate the model
y_true_all, y_pred_all = eval_sklearn_model(
    vectorizer=bow
    ,classifier=knn
    ,skf=skf
    ,X_train=X_train
    ,y_train=y_train
)

In [ ]:
print("Overall classification report across all folds:\n")
print(classification_report(y_true_all, y_pred_all, digits=4, target_names=LABELS.keys()))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_true_all, y_pred_all)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=LABELS.keys(), yticklabels=LABELS.keys())
plt.xlabel('Predicted', labelpad=15)
plt.ylabel('True', labelpad=15)
plt.title('Confusion Matrix (KNN Bag of Words)', fontsize=16)
plt.show()

#### Word2Vec

In [ ]:
# Evaluate the model
y_true_all, y_pred_all = eval_sklearn_model(
    vectorizer=w2v
    ,classifier=knn
    ,skf=skf
    ,X_train=X_train
    ,y_train=y_train
)

In [ ]:
print("Overall classification report across all folds:\n")
print(classification_report(y_true_all, y_pred_all, digits=4, target_names=LABELS.keys()))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_true_all, y_pred_all)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=LABELS.keys(), yticklabels=LABELS.keys())
plt.xlabel('Predicted', labelpad=15)
plt.ylabel('True', labelpad=15)
plt.title('Confusion Matrix (KNN Word2Vec)', fontsize=16)
plt.show()

#### Transformer

In [ ]:
# Evaluate the model
y_true_all, y_pred_all = eval_transformer(
    transformer=transf_model
    ,objective='feature-extraction'
    ,skf=skf
    ,X_train=X_train
    ,y_train=y_train
    ,classifier=knn
)

In [ ]:
print("Overall classification report across all folds:\n")
print(classification_report(y_true_all, y_pred_all, digits=4, target_names=LABELS.keys()))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_true_all, y_pred_all)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=LABELS.keys(), yticklabels=LABELS.keys())
plt.xlabel('Predicted', labelpad=15)
plt.ylabel('True', labelpad=15)
plt.title('Confusion Matrix (KNN Transformer)', fontsize=16)
plt.show()

### 5.2. LSTM

In [ ]:
y_true_all, y_pred_all, lstm_history = eval_lstm_model(
    vectorizer=w2v_model
    ,emb_size=emb_size
    ,skf=skf
    ,X_train=X_train
    ,y_train=y_train
)

In [ ]:
print("Overall classification report across all folds:\n")
print(classification_report(y_true_all, y_pred_all, digits=4, target_names=LABELS.keys()))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_true_all, y_pred_all)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=LABELS.keys(), yticklabels=LABELS.keys())
plt.xlabel('Predicted', labelpad=15)
plt.ylabel('True', labelpad=15)
plt.title('Confusion Matrix (LSTM)', fontsize=16)
plt.show()

In [ ]:
df_list = []

for fold_idx, history in enumerate(lstm_history):
    df_fold = pd.DataFrame(history.history)
    df_fold['epoch'] = range(1, len(df_fold) + 1)
    df_fold['fold'] = fold_idx  # add fold index column
    # Reorder columns so epoch and fold come first (optional)
    cols = ['fold', 'epoch'] + [col for col in df_fold.columns if col not in ('fold', 'epoch')]
    df_fold = df_fold[cols]
    df_list.append(df_fold)

# Concatenate all folds into a single DataFrame
df_history = pd.concat(df_list, ignore_index=True)

# Show head
df_history.head()

In [ ]:
plt.figure(figsize=(12, 8))

epochs = sorted(df_history['epoch'].unique())
train_means = []
val_means = []
train_stds = []
val_stds = []

for epoch in epochs:
    epoch_data = df_history[df_history['epoch'] == epoch]
    train_acc = epoch_data['accuracy']
    val_acc = epoch_data['val_accuracy']
    
    train_means.append(train_acc.mean())
    val_means.append(val_acc.mean())
    train_stds.append(train_acc.std())
    val_stds.append(val_acc.std())

train_means = np.array(train_means)
val_means = np.array(val_means)
train_stds = np.array(train_stds)
val_stds = np.array(val_stds)

plt.plot(epochs, train_means, label='Train Mean Accuracy')
plt.fill_between(epochs, train_means - train_stds, train_means + train_stds, alpha=0.2)

plt.plot(epochs, val_means, label='Val Mean Accuracy')
plt.fill_between(epochs, val_means - val_stds, val_means + val_stds, alpha=0.2)

# Calculate min and max across both train and val means
y_min = min(min(train_means), min(val_means)) - 0.1
y_max = max(max(train_means), max(val_means)) + 0.1

# Prevent y_min from going below 0
y_min = max(y_min, 0)

plt.ylim(y_min, y_max)

plt.title('Mean Model Accuracy Across Folds')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

### 5.3. Encoder

In [ ]:
# Evaluate the model
y_true_all, y_pred_all = eval_transformer(
    transformer=transf_model
    ,objective='sentiment-analysis'
    ,skf=skf
    ,X_train=X_train
    ,y_train=y_train
)

In [ ]:
print("Overall classification report across all folds:\n")
print(classification_report(y_true_all, y_pred_all, digits=4, target_names=LABELS.keys()))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_true_all, y_pred_all)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=LABELS.keys(), yticklabels=LABELS.keys())
plt.xlabel('Predicted', labelpad=15)
plt.ylabel('True', labelpad=15)
plt.title('Confusion Matrix (KNN Transformer)', fontsize=16)
plt.show()

### 5.4. Language Model

In [ ]:
system_message = """
You are a helpful assistant that performs sentiment analysis.

Your task is to analyze the overall sentiment of a given sentence or text.

For each input, classify the sentiment as one of the following single digits ONLY:
- 0: Bearish (negative sentiment)
- 1: Bullish (positive sentiment)
- 2: Neutral sentiment

Respond ONLY with a single digit (0, 1, or 2), with no extra text or explanation.

Examples:
- 'JPMorgan reels in expectations on Beyond Meat' → 0
- 'Dougherty & Company starts at Buy' → 1
- 'Analysts React To FCC Decision On Intelsat C-Band Spectrum Auction' → 2
"""

In [ ]:
# Evaluate the model
y_true_all, y_pred_all = eval_llm_model(
    model="tiiuae/falcon-7b-instruct"
    ,skf=skf
    ,X_train=X_train
    ,y_train=y_train
    ,system_message=system_message
)

In [ ]:
print("Overall classification report across all folds:\n")
print(classification_report(y_true_all, y_pred_all, digits=4, target_names=LABELS.keys()))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_true_all, y_pred_all)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=LABELS.keys(), yticklabels=LABELS.keys())
plt.xlabel('Predicted', labelpad=15)
plt.ylabel('True', labelpad=15)
plt.title('Confusion Matrix (KNN Transformer)', fontsize=16)
plt.show()

In [ ]:
# TODO: Review code; Complete aux_funcs.py 'analyze_sentiment'; Implement extra feature engineering (for extra points); Implement extra classification model